The purpose of this analysis is to examine the efficacy of college standardized admissions tests and whether they're unfair to particular demographics. As such, we will be investigating the potential correlation between SAT/AP test scores and various demographic factors, such as race, gender, and income. 

Data was taken from the NYC Open Data website concerning NYC high schools. The files are as follows:

    * 'ap_scores' - Data on AP test scores
    * 'class_size' - Data on class size
    * 'demographics' - Data on class high school demographics
    * 'graduation_outcomes' - Data on graduation rates, etc.
    * 'hs_directory' - A general directory of the high schools
    * 'sat_results' - Data on SAT scores
    * 'all_survey' - Data on surveys from all NYC high schools
    * 'd75_survey' - Data on surveys from NYC district 75
   
Currently, this file is an exploration in data cleaning. Analysis of the data is ongoing.

In [58]:
import pandas as pd
import numpy as np
data ={}

## We'll read in the .csv files first. Survey results will be read in during the following step using a different pandas tool.
file_list = ['ap_scores.csv','hs_directory.csv','class_size.csv','demographics.csv','graduation_outcomes.csv','sat_results.csv']
print(sorted(file_list))
for file_name in sorted(file_list):
    string_name = str.replace(file_name,'.csv','')
    data[string_name] = pd.read_csv(file_name)
    print(data[string_name].columns)
    
print(data['ap_scores'].loc[0,:])
print(data['class_size'].loc[0,:])

['ap_scores.csv', 'class_size.csv', 'demographics.csv', 'graduation_outcomes.csv', 'hs_directory.csv', 'sat_results.csv']
Index(['DBN', 'SchoolName', 'AP Test Takers ', 'Total Exams Taken',
       'Number of Exams with scores 3 4 or 5'],
      dtype='object')
Index(['CSD', 'BOROUGH', 'SCHOOL CODE', 'SCHOOL NAME', 'GRADE ',
       'PROGRAM TYPE', 'CORE SUBJECT (MS CORE and 9-12 ONLY)',
       'CORE COURSE (MS CORE and 9-12 ONLY)', 'SERVICE CATEGORY(K-9* ONLY)',
       'NUMBER OF STUDENTS / SEATS FILLED', 'NUMBER OF SECTIONS',
       'AVERAGE CLASS SIZE', 'SIZE OF SMALLEST CLASS', 'SIZE OF LARGEST CLASS',
       'DATA SOURCE', 'SCHOOLWIDE PUPIL-TEACHER RATIO'],
      dtype='object')
Index(['DBN', 'Name', 'schoolyear', 'fl_percent', 'frl_percent',
       'total_enrollment', 'prek', 'k', 'grade1', 'grade2', 'grade3', 'grade4',
       'grade5', 'grade6', 'grade7', 'grade8', 'grade9', 'grade10', 'grade11',
       'grade12', 'ell_num', 'ell_percent', 'sped_num', 'sped_percent',
       'ctt_nu

So far, each of the data sets appear to have a 'DBN' column (with the exception of 'class_size' and 'hs_directory'), which provides a unique identifer for each of the schools. We can use this as we clean and merge the data sets. 

In [59]:
# Now that we've read in our .csv files, we can read in the two survey files, which are .xlxs files. We should also merge them
# together to save time as we add them to our 'data' dictionary. 

all_survey = pd.io.excel.read_excel('all_survey.xlsx')
d75_survey = pd.io.excel.read_excel('d75_survey.xlsx')
survey = pd.concat([all_survey,d75_survey],axis=0)

In [60]:
# The survey data contains far more columns of information than we need for our analysis. Rather than adding the entire data set
# into our dictionary, we can examine the data dictionary that accompanied the data sets to see what is most useful to us. After
# a look at the dictionary, it seems as though the following columns are most relevant.
columns = ["DBN", "rr_s", "rr_t", "rr_p", "N_s", "N_t", "N_p", "saf_p_11", "com_p_11", "eng_p_11", "aca_p_11", "saf_t_11", "com_t_11", "eng_t_11", "aca_t_11", "saf_s_11", "com_s_11", "eng_s_11", "aca_s_11", "saf_tot_11", "com_tot_11", "eng_tot_11", "aca_tot_11"]
survey['DBN'] = survey['dbn']

survey = survey[columns]
data['survey'] = survey
print(data['survey'].shape)
print(data['survey'].head(1))

(1702, 23)
      DBN  rr_s  rr_t  rr_p  N_s   N_t   N_p  saf_p_11  com_p_11  eng_p_11  \
0  01M015   NaN    88    60  NaN  22.0  90.0       8.5       7.6       7.5   

      ...      eng_t_11  aca_t_11  saf_s_11  com_s_11  eng_s_11  aca_s_11  \
0     ...           7.6       7.9       NaN       NaN       NaN       NaN   

   saf_tot_11  com_tot_11  eng_tot_11  aca_tot_11  
0         8.0         7.7         7.5         7.9  

[1 rows x 23 columns]


In [61]:
# There are two more steps we can take to make data merging and analysis easier: (1) we can create DBN columns for class_size 
# and hs_directory, and (2) we can combine the SAT scores so we only have the aggregate SAT score. 

## Create 'DBN' column for hs_directory.
data['hs_directory']['DBN'] = data['hs_directory']['dbn']

## A school's DBN in class_size seems to be a combination of it's CSD and School Code, albeit with an extra '0' in front. To help
## us complete the merge, we can use this info to create a DBN column for class_size.
def create_dbn(csd):
    string_csd = str(csd)
    if len(string_csd) < 2:
        return string_csd.zfill(2)
    else:
        return string_csd

padded_csd = data['class_size']['CSD'].apply(create_dbn)
data['class_size']['DBN'] = padded_csd + data['class_size']['SCHOOL CODE']
data['class_size']['DBN'].head()

0    01M015
1    01M015
2    01M015
3    01M015
4    01M015
Name: DBN, dtype: object

In [62]:
## Now we can calulate total average SAT score for each school. First, we need to transform each column in integers for calculations.
sat_sections = ['SAT Math Avg. Score','SAT Critical Reading Avg. Score', 'SAT Writing Avg. Score']
for each in sat_sections:
    data['sat_results'][each] = pd.to_numeric(data['sat_results'][each], errors='coerce')

data['sat_results']['sat_score'] = data['sat_results']['SAT Math Avg. Score'] + data['sat_results']['SAT Critical Reading Avg. Score'] + data['sat_results']['SAT Writing Avg. Score']
data['sat_results']['sat_score'].head()

0    1122.0
1    1172.0
2    1149.0
3    1174.0
4    1207.0
Name: sat_score, dtype: float64

In [63]:
## Before moving onto the merge, let's parse the geographic info from the hs_directory data set. This will allow us to geographically
## plot the high schools if we desire. The latitude and longitude is contained within 'Location 1,' but we'll need to extract it.
print(data['hs_directory']['Location 1'].head(1))
import re
def find_lat(line):
    coordinates = re.findall('\(.+\)',line)
    return coordinates[0].split(',')[0].replace('(','')
def find_long(line):
    coordinates = re.findall('\(.+\)',line)
    return coordinates[0].split(',')[1].replace(')','')

data['hs_directory']['lat'] = pd.to_numeric(data['hs_directory']['Location 1'].apply(find_lat),errors='coerce')
data['hs_directory']['lon'] = pd.to_numeric(data['hs_directory']['Location 1'].apply(find_long),errors='coerce')

print(data['hs_directory'].head(3))

0    883 Classon Avenue\nBrooklyn, NY 11225\n(40.67...
Name: Location 1, dtype: object
      dbn                                  school_name   borough  \
0  17K548          Brooklyn School for Music & Theatre  Brooklyn   
1  09X543             High School for Violin and Dance     Bronx   
2  09X327  Comprehensive Model School Project M.S. 327     Bronx   

  building_code  phone_number    fax_number  grade_span_min  grade_span_max  \
0          K440  718-230-6250  718-230-6262             9.0              12   
1          X400  718-842-0687  718-589-9849             9.0              12   
2          X240  718-294-8111  718-294-8109             6.0              12   

   expgrade_span_min  expgrade_span_max    ...      \
0                NaN                NaN    ...       
1                NaN                NaN    ...       
2                NaN                NaN    ...       

                                          Location 1 Community Board  \
0  883 Classon Avenue\nBrooklyn, N

Now that we've manipulated the data sets to contain the proper columns necessary for analysis, we are almost ready to proceed with the merge. However, there are still multiple data sets that contain repeat 'DBN' values; since we plan on using the 'DBN' column to merge the data sets, this will be problematic. We'll need to condense the data sets before we continue onward to minimize data loss. We'll also need to finish up some last minute preparations (e.g. transforming strings to numerical values) so that future analysis becomes easier.

In [64]:
## First, let's examine any discrepancies across the number of rows between the data sets.
for each in data:
    print(each, data[each].shape[0],data[each].columns)

ap_scores 258 Index(['DBN', 'SchoolName', 'AP Test Takers ', 'Total Exams Taken',
       'Number of Exams with scores 3 4 or 5'],
      dtype='object')
class_size 27611 Index(['CSD', 'BOROUGH', 'SCHOOL CODE', 'SCHOOL NAME', 'GRADE ',
       'PROGRAM TYPE', 'CORE SUBJECT (MS CORE and 9-12 ONLY)',
       'CORE COURSE (MS CORE and 9-12 ONLY)', 'SERVICE CATEGORY(K-9* ONLY)',
       'NUMBER OF STUDENTS / SEATS FILLED', 'NUMBER OF SECTIONS',
       'AVERAGE CLASS SIZE', 'SIZE OF SMALLEST CLASS', 'SIZE OF LARGEST CLASS',
       'DATA SOURCE', 'SCHOOLWIDE PUPIL-TEACHER RATIO', 'DBN'],
      dtype='object')
demographics 10075 Index(['DBN', 'Name', 'schoolyear', 'fl_percent', 'frl_percent',
       'total_enrollment', 'prek', 'k', 'grade1', 'grade2', 'grade3', 'grade4',
       'grade5', 'grade6', 'grade7', 'grade8', 'grade9', 'grade10', 'grade11',
       'grade12', 'ell_num', 'ell_percent', 'sped_num', 'sped_percent',
       'ctt_num', 'selfcontained_num', 'asian_num', 'asian_per', 'black_num',
 

In [65]:
## Condense class_size by filtering for high school grades in 'GRADE' only. We can also filter 'PROGRAM TYPE' down to 'GEN ED'
## because it's the largest program type by far.
print(data['class_size']['GRADE '].unique())
print(data['class_size']['PROGRAM TYPE'].unique())
class_size = data['class_size']
class_size = class_size[class_size['GRADE '] == '09-12']
class_size = class_size[class_size['PROGRAM TYPE'] == 'GEN ED']
data['class_size'] = class_size

## Since there still replicate DBNs, we will need to filter class_size down even further. Since a single DBN appears to correspond
## with multiple core subjects, we need to group the data set by DBN and aggregate the data in this column by average.
class_size = class_size.groupby('DBN').aggregate(np.mean)
class_size.reset_index(inplace=True)
data['class_size'] = class_size
print(data['class_size'].head())

['0K' '01' '02' '03' '04' '05' '0K-09' nan '06' '07' '08' 'MS Core' '09-12'
 '09']
['GEN ED' 'CTT' 'SPEC ED' nan 'G&T']
      DBN  CSD  NUMBER OF STUDENTS / SEATS FILLED  NUMBER OF SECTIONS  \
0  01M292    1                            88.0000            4.000000   
1  01M332    1                            46.0000            2.000000   
2  01M378    1                            33.0000            1.000000   
3  01M448    1                           105.6875            4.750000   
4  01M450    1                            57.6000            2.733333   

   AVERAGE CLASS SIZE  SIZE OF SMALLEST CLASS  SIZE OF LARGEST CLASS  \
0           22.564286                   18.50              26.571429   
1           22.000000                   21.00              23.500000   
2           33.000000                   33.00              33.000000   
3           22.231250                   18.25              27.062500   
4           21.200000                   19.40              22.866667   

   SCHOO

In [66]:
## Now we can condense demographics. Since the only thing that's duplicating DBNs in this
## data set is 'schoolyear', we can select only the most recent school year (2011-2012) for filtration.
data['demographics'] = data['demographics'][data['demographics']['schoolyear'] == 20112012]
data['demographics'].head()

,DBN,Name,schoolyear,fl_percent,frl_percent,total_enrollment,prek,k,grade1,grade2,...,black_num,black_per,hispanic_num,hispanic_per,white_num,white_per,male_num,male_per,female_num,female_per
6,01M015,P.S. 015 ROBERTO CLEMENTE,20112012,NaN,89.4,189,13,31,35,28,...,63,33.3,109,57.7,4,2.1,97.0,51.3,92.0,48.7
13,01M019,P.S. 019 ASHER LEVY,20112012,NaN,61.5,328,32,46,52,54,...,81,24.7,158,48.2,28,8.5,147.0,44.8,181.0,55.2
20,01M020,PS 020 ANNA SILVER,20112012,NaN,92.5,626,52,102,121,87,...,55,8.8,357,57.0,16,2.6,330.0,52.7,296.0,47.3
27,01M034,PS 034 FRANKLIN D ROOSEVELT,20112012,NaN,99.7,401,14,34,38,36,...,90,22.4,275,68.6,8,2.0,204.0,50.9,197.0,49.1
35,01M063,PS 063 WILLIAM MCKINLEY,20112012,NaN,78.9,176,18,20,30,21,...,41,23.3,110,62.5,15,8.5,97.0,55.1,79.0,44.9


In [72]:
## Finally, we can condense the graduation data set. In this data set, it's the 'Demographic' and 'Cohort' columns causing 
## replicate DBNs to appear.
data['graduation_outcomes']['Demographic'].unique,data['graduation_outcomes']['Cohort'].unique
## It appears that we want to include only columns with 'Total Cohort' for 'Demographic' and '2006' (the most recent cohort)
## for 'Cohort.'
data['graduation_outcomes'] = data['graduation_outcomes'][data['graduation_outcomes']['Demographic'] == 'Total Cohort']
data['graduation_outcomes'] = data['graduation_outcomes'][data['graduation_outcomes']['Cohort'] == '2006']

In [80]:
## Before merging the data sets, let's take a second to convert the strings in ap_scores to numeric values for easier analysis.
data['ap_scores'].columns
columns_to_convert = ['AP Test Takers ','Total Exams Taken','Number of Exams with scores 3 4 or 5']
for column in columns_to_convert:
    data['ap_scores'][column] = pd.to_numeric(data['ap_scores'][column],errors='coerce')

In [88]:
dataframes = ['sat_results','ap_scores','graduation_outcomes','class_size','demographics','survey','hs_directory']

for each in dataframes:
    print(each,data[each].shape)

sat_results (478, 7)
ap_scores (258, 5)
graduation_outcomes (405, 23)
class_size (583, 8)
demographics (1509, 38)
survey (1702, 23)
hs_directory (435, 67)


Now that we've reasonably condensed each data set and finished preparations, we can finally continue on with the merge. Because our primary question concerns the correlation between test scores and demographics, we'll want to follow these merge strategies:
 
    * Because sat_results is the data set we're primarily interested in, we'll use it as our starter data set for the merge.
    * Because ap_scores and graduation_outcomes have missing DBN values in relation to sat_results, we'll perform a left 
      merge. This allows us to keep the DBN values from sat_results while adding info from these data sets.
    * Because the other data sets, class_size, demographics, survey, and hs_directory, all contain valuable information that   we don't want to lose, we'll perform an inner merge.
    

In [77]:
combined = data['sat_results']
combined = combined.merge(data['ap_scores'],how='left',on='DBN')
combined = combined.merge(data['graduation_outcomes'],how='left',on='DBN')
combined.shape

(479, 33)

In [79]:
to_merge = ['class_size', 'demographics','survey','hs_directory']
for df in to_merge:
    combined = combined.merge(data[df],how='inner',on='DBN')
combined.shape

(363, 297)

In [89]:
## Now that we've merged all the data sets, we can address rows with missing data from the merge using the df.fillna() method. 
## To simplify things, let's go ahead and just fill in the missing values with the mean for the entire column or 0 for the rest.
combined = combined.fillna(combined.mean())
combined = combined.fillna(0)